# Learning from Projections

In [ ]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

## Data loading and preprocessing

In [ ]:
# Getting the files (too large this time to store in repository)
!wget -nc https://graphics.stanford.edu/data/3Dscanrep/xyzrgb/xyzrgb_dragon.ply.gz
!gunzip -n xyzrgb_dragon.ply.gz  # Interrupt kernel if asked if you want to overwrite existing file

# Load PLY file and reduce density to 1%
pcd = o3d.io.read_point_cloud("xyzrgb_dragon.ply")
pcd = pcd.uniform_down_sample(100)

# Transform point cloud to Numpy array, remove outliers, move away from camera
points = np.asarray(pcd.points)
points = points[points[:, 1] > -42]
x = points[:, 0]
y = points[:, 1]
z = points[:, 2]
z -= z.min()
z += 125

object3d = go.Scatter3d(x=x,
                        y=y,
                        z=z,
                        mode='markers',
                        marker=dict(size=1,
                                    color=y,
                                    colorscale='magma'),
                        hovertemplate="<b>Point</b><br>x: %{x:.2f}<br>y: %{y:.2f}<br>z: %{z:.2f}<extra></extra>",
                        showlegend=False)

fig = go.Figure(object3d)

fig.layout.scene.camera=dict(eye=dict(x=0, y=0, z=-1.5),
                             up=dict(x=0, y=1, z=0))

fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  showlegend=False,
                  width=640,
                  height=480,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_dragmode='orbit')

## Projection using pinhole camera model

In [ ]:
# Camera intrinsics (taken from Kinect RGB-D camera)
width = 640
height = 480
principal_point_offset = [319.5, 239.5]
focal_length = 525.0
axis_skew = 0

f_x = focal_length
f_y = focal_length
x_0 = principal_point_offset[0]
y_0 = principal_point_offset[1]
s = axis_skew

camera_intriniscs = np.array([[f_x, s, x_0],
                              [0, f_y, y_0],
                              [0,   0, 1]])
K = camera_intriniscs

def pinhole_camera_projection(point, camera_intrinsics):
    K = camera_intrinsics
    point_x, point_y, point_z = point
    pixel_x = np.rint(point_x * K[0, 0] / point_z + K[0, 2]).astype(np.int32)
    pixel_y = np.rint(point_y * K[1, 1] / point_z + K[1, 2]).astype(np.int32)
    return pixel_x, pixel_y

pixels = np.array([pinhole_camera_projection(p, K) for p in points])

# Can be used later to make an orthographic projection of
# the image pixels back into 3D space
points_z = points[:, 2]
points_z = points_z[(pixels[:, 0] >= 0) & (pixels[:, 1] >= 0)]
points_z = points_z[(pixels[:, 0] < width) & (pixels[:, 1] < height)]

# Add intensity values to pixels and remove erroneous/out of view projections
pixels = np.vstack([pixels.T, y]).T
pixels = pixels[(pixels[:, 0] >= 0) & (pixels[:, 1] >= 0)]
pixels = pixels[(pixels[:, 0] < width) & (pixels[:, 1] < height)]

## 2D projection

In [ ]:
image2d = go.Scattergl(x=pixels[:, 0],
                       y=pixels[:, 1],
                       mode='markers',
                       marker=dict(size=1,
                                   color=pixels[:, 2],
                                   colorscale='magma',
                                   symbol='square'),
                       hovertemplate="<b>Pixel:</b> %{x}, %{y}<br><extra></extra>",
                       showlegend=False)

fig = go.Figure(image2d)

fig.layout.scene.camera=dict(eye=dict(x=0, y=0, z=-1),
                             up=dict(x=0, y=-1, z=0),
                             projection=dict(type="orthographic"))

fig.update_layout(template='plotly_white',
                  xaxis=dict(constrain='domain',
                             visible=False,
                             autorange='reversed'),
                  yaxis=dict(scaleanchor='x',
                             visible=False),
                  hoverlabel=dict(font_size=18),
                  width=width,
                  height=height,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0))

## 2D projection in 3D space

In [ ]:
# Replace 80 by points_z to obtain orthographic projection
x = (pixels[:, 0] - K[0, 2]) / K[0, 0] * 80
y = (pixels[:, 1] - K[1, 2]) / K[1, 1] * 80
z = np.ones_like(pixels[:, 0]) * 80  # Distance from camera

image3d = go.Scatter3d(x=x,
                       y=y,
                       z=z,
                       mode='markers',
                       marker=dict(size=1,
                                   color=y,
                                   colorscale='magma',
                                   symbol='square'),
                       hoverinfo='none',
                       showlegend=False)

fig = go.Figure(image3d)

fig.layout.scene.camera=dict(eye=dict(x=0, y=0, z=-1),
                             up=dict(x=0, y=1, z=0),
                             projection=dict(type='orthographic'))

fig.update_layout(template='plotly_white',
                  scene=dict(xaxis=dict(visible=False),
                             yaxis=dict(visible=False,),
                             zaxis=dict(visible=False),
                             aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  width=width,
                  height=height,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0))

## Comparing 3D object and 2D projection (in 3D space)

In [ ]:
fig = make_subplots(rows=1,
                    cols=2,
                    horizontal_spacing=0,
                    vertical_spacing=0,
                    specs=[[dict(type='scene'), dict(type='scene')]])

fig.add_trace(object3d, row=1, col=1)
fig.add_trace(image3d, row=1, col=2)

fig.layout.scene.camera=dict(eye=dict(x=0, y=0, z=-2.5),
                             up=dict(x=0, y=1, z=0))
fig.layout.scene2.camera=dict(eye=dict(x=0, y=0, z=-9),
                              up=dict(x=0, y=1, z=0))

fig.update_layout(template='plotly_white',
                  scene=dict(xaxis=dict(visible=False),
                             yaxis=dict(visible=False),
                             zaxis=dict(visible=False),
                             aspectmode='data'),
                  scene2=dict(xaxis=dict(visible=False),
                              yaxis=dict(visible=False),
                              zaxis=dict(visible=False),
                              aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  height=400,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0))

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/dragon_3d_2d.html",
               full_html=False,
               include_plotlyjs='cdn')

## Putting it all together

In [ ]:
fig = go.Figure([image3d, object3d])

# The viewpoint and field of view
eye = go.Scatter3d(x=[0], y=[0], z=[0], marker=dict(size=10, color='black'), name="eye", hoverinfo='name')
view = []
view.append(go.Scatter3d(x=[0, 170], y=[0, 90], z=[0, 270], mode='lines', marker_color='red', hoverinfo='none'))
view.append(go.Scatter3d(x=[0, 170], y=[0, -90], z=[0, 270], mode='lines', marker_color='red', hoverinfo='none'))
view.append(go.Scatter3d(x=[0, -170], y=[0, 90], z=[0, 270], mode='lines', marker_color='red', hoverinfo='none'))
view.append(go.Scatter3d(x=[0, -170], y=[0, -90], z=[0, 270], mode='lines', marker_color='red', hoverinfo='none'))

view.append(go.Scatter3d(x=[-170, 170], y=[90, 90], z=[270, 270], mode='lines', marker_color='red', hoverinfo='none'))
view.append(go.Scatter3d(x=[-170, 170], y=[-90, -90], z=[270, 270], mode='lines', marker_color='red', hoverinfo='none'))
view.append(go.Scatter3d(x=[170, 170], y=[-90, 90], z=[270, 270], mode='lines', marker_color='red', hoverinfo='none'))
view.append(go.Scatter3d(x=[-170, -170], y=[-90, 90], z=[270, 270], mode='lines', marker_color='red', hoverinfo='none'))

view.append(go.Scatter3d(x=[-50.37, 50.37], y=[26.67, 26.67], z=[80, 80], mode='lines', marker_color='black',
                         hovertemplate="<b>Image</b><extra></extra>"))
view.append(go.Scatter3d(x=[-50.37, 50.37], y=[-26.67, -26.67], z=[80, 80], mode='lines', marker_color='black',
                         hovertemplate="<b>Image</b><extra></extra>"))
view.append(go.Scatter3d(x=[50.37, 50.37], y=[-26.67, 26.67], z=[80, 80], mode='lines', marker_color='black',
                         hovertemplate="<b>Image</b><extra></extra>"))
view.append(go.Scatter3d(x=[-50.37, -50.37], y=[-26.67, 26.67], z=[80, 80], mode='lines', marker_color='black',
                         hovertemplate="<b>Image</b><extra></extra>"))

view.append(go.Scatter3d(x=[0, -21.94], y=[0, 19.14], z=[0, 168.49], mode='markers+lines', marker_color='lime',
                         hovertemplate="<b>Projection</b><extra></extra>", marker_size=2))
view.append(eye)

fig.add_traces(view)

fig.layout.scene.camera=dict(eye=dict(x=-1, y=1, z=-1),
                             up=dict(x=0, y=1, z=0),
                             center=dict(x=-0.3, y=0.2))

fig.update_layout(template='plotly_white',
                  scene=dict(xaxis=dict(visible=False),
                             yaxis=dict(visible=False),
                             zaxis=dict(visible=False),
                             aspectmode='data'),
                  hoverlabel=dict(font_size=18),
                  showlegend=False,
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0))

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/projection.html",
               full_html=False,
               include_plotlyjs='cdn')